In [3]:
!open .

In [4]:
import xlwings as xw
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_rows', 1000)


def KBpriceindex_preprocessing(path, data_type):
    # path : KB 데이터 엑셀 파일의 디렉토리 (문자열)
    # data_type : ‘매매종합’, ‘매매APT’, ‘매매연립’, ‘매매단독’, ‘전세종합’, ‘전세APT’, ‘전세연립’, ‘전세단독’ 중 하나
    
    wb = xw.Book(path)                
    sheet = wb.sheets[data_type]   
    row_num = sheet.range((1,1)).end('down').end('down').end('down').row  
    data_range = 'A2:GE' + str(row_num)
    raw_data = sheet[data_range].options(pd.DataFrame, index=False, header=True).value 
    
    bignames = '서울 대구 부산 대전 광주 인천 울산 세종 경기 강원 충북 충남 전북 전남 경북 경남 제주도 6개광역시 5개광역시 수도권 기타지방 구분 전국'
    bigname_list = bignames.split(' ')
    big_col = list(raw_data.columns)
    small_col = list(raw_data.iloc[0])

    for num, gu_data in enumerate(small_col):
        if gu_data == None:
            small_col[num] = big_col[num]

        check = num
        while True:
            if big_col[check] in bigname_list:
                big_col[num] = big_col[check]
                break
            else:
                check = check - 1
                
    big_col[129] = '경기' 
    big_col[130] = '경기'
    small_col[185] = '서귀포'
    
    raw_data.columns = [big_col, small_col]
    new_col_data = raw_data.drop([0,1])
    
    index_list = list(new_col_data['구분']['구분'])

    new_index = []

    for num, raw_index in enumerate(index_list):
        temp = str(raw_index).split('.')
        if int(temp[0]) > 20 :
            if len(temp[0]) == 2:
                new_index.append('19' + temp[0] + '.' + temp[1])
            else:
                new_index.append(temp[0] + '.' + temp[1])
        else:
            new_index.append(new_index[num-1].split('.')[0] + '.' + temp[0])

    new_col_data.set_index(pd.to_datetime(new_index), inplace=True)
    cleaned_data  = new_col_data.drop(('구분', '구분'), axis=1)
    return cleaned_data

path = r'data/★(월간)KB주택가격동향_시계열(2019.09).xlsx'
price_index = KBpriceindex_preprocessing(path, '매매apt')
jeonse_index = KBpriceindex_preprocessing(path, '전세apt')

index_date = datetime(2019, 9, 1)
prev_date = datetime(2019, 1, 1)
# time_range = 12 * 15
# prev_date = index_date - relativedelta(months=time_range)

print(index_date)
print(prev_date)

index_date = datetime(2019, 1, 1)
time_range = 12 * 15
prev_date = index_date - relativedelta(months=time_range)

demand_df = pd.DataFrame()
demand_df['매매증감률'] = (price_index.loc[index_date] - price_index.loc[prev_date])/ price_index.loc[prev_date]
demand_df['전세증감률'] = (jeonse_index.loc[index_date] - jeonse_index.loc[prev_date])/jeonse_index.loc[prev_date]
demand_df = demand_df * 100
demand_df['매매버블'] = demand_df['매매증감률'] - demand_df['매매증감률'][('전국','전국')]
demand_df['전세버블'] = demand_df['전세증감률'] - demand_df['전세증감률'][('전국','전국')]
demand_df['버블'] = demand_df['매매버블'] -  demand_df['전세버블']
sorted_df = demand_df
sorted_df = demand_df.sort_values(by='버블')
# sorted_df['rank'] = demand_df['버블'].rank()
# sorted_df = sorted_df.loc['서울']
sorted_df['rank'] = sorted_df['버블'].rank()
sorted_df.dropna(inplace=True)
sorted_df


2019-09-01 00:00:00
2019-01-01 00:00:00


매매증감률    전세증감률       매매버블       전세버블        버블   rank
경기    용인     43.1773  139.173   -20.1143    50.1354  -70.2497    1.0
      남양주    33.5993   127.19   -29.6924    38.1526   -67.845    2.0
      화성     25.1866  88.9682    -38.105 -0.0691461  -38.0359    3.0
      분당구    66.9213  130.507    3.62964    41.4695  -37.8398    4.0
      단원구      49.81   109.82   -13.4817    20.7831  -34.2648    5.0
      양주     16.5555  76.5096   -46.7361   -12.5277  -34.2084    6.0
      일산서구   43.3672  102.421   -19.9245    13.3838  -33.3083    7.0
      김포     19.8965  77.1704   -43.3951    -11.867  -31.5281    8.0
      구리     66.7362  123.784    3.44454    34.7469  -31.3023    9.0
      고양     44.2622  101.189   -19.0295    12.1516  -31.1811   10.0
      일산동구   40.4936  95.9152    -22.798    6.87782  -29.6758   11.0
      성남     63.5904  118.815   0.298716    29.7781  -29.4794   12.0
서울    강북구    56.4021  111.441   -6.88951    22.4041  -29.2936   13.0
경기    장안구    40.1464  95.0947   -23.1453    6.05736  -29.2026   14.0
      덕양구    46.6398  101.287   -16.6519      12.25  -28.9019   15.0
      의정부    37.6088  91.8344   -25.6828    2.79703  -28.4799   16.0
      경기     49.3407  102.866   -13.9509    13.8286  -27.7795   17.0
      안산     55.3041   108.57   -7.98757    19.5326  -27.5202   18.0
      수원     43.9473  95.1552   -19.3443    6.11788  -25.4622   19.0
서울    강서구    70.9434  121.425    7.65181    32.3874  -24.7356   20.0
대전    유성구    31.5997  81.1085    -31.692   -7.92886  -23.7631   21.0
경기    광주     38.9137  88.0063   -24.3779   -1.03107  -23.3469   22.0
      오산     28.0126  75.6813   -35.2791   -13.3561   -21.923   23.0
      하남     39.9367  86.4025    -23.355   -2.63486  -20.7201   24.0
충남    아산     32.8935  79.3338   -30.3982   -9.70359  -20.6946   25.0
경기    부천     44.9285  91.2333   -18.3632    2.19598  -20.5591   26.0
서울    양천구    78.8982  124.882    15.6065    35.8442  -20.2377   27.0
경기    권선구    39.3636  84.9618   -23.9281   -4.07558  -19.8525   28.0
서울    성북구    61.8694  107.427   -1.42225    18.3893  -19.8116   29.0
경기    상록구    60.8774   105.84    -2.4142     16.803  -19.2172   30.0
      동안구    85.3331  129.238    22.0415    40.2006  -18.1591   31.0
      파주     29.8197  73.6375    -33.472   -15.3999  -18.0721   32.0
서울    중랑구    56.2286  100.017   -7.06301    10.9796  -18.0426   33.0
      구로구    72.5099  116.198    9.21823    27.1607  -17.9425   34.0
충남    논산     63.3549  106.986  0.0632972    17.9483   -17.885   35.0
인천    부평구    34.6862  77.8702   -28.6055   -11.1671  -17.4383   36.0
경기    안양     79.1652  121.232    15.8735    32.1943  -16.3207   37.0
인천    연수구    33.5474   75.325   -29.7442   -13.7123  -16.0319   38.0
서울    마포구    82.9441  124.499    19.6524    35.4613  -15.8089   39.0
수도권   수도권    57.3446  98.4035     -5.947     9.3661  -15.3131   40.0
서울    관악구    68.5247   109.35     5.2331    20.3124  -15.0793   41.0
충남    충남     35.5126  76.0791   -27.7791   -12.9583  -14.8208   42.0
서울    광진구    66.8691  107.005    3.57743    17.9678  -14.3904   43.0
경기    군포     77.9941  117.903    14.7025    28.8658  -14.1634   44.0
      만안구    59.3222  98.9813   -3.96947    9.94395  -13.9134   45.0
      의왕     81.7063  121.279    18.4147    32.2412  -13.8265   46.0
충북    충주     57.8603    97.35   -5.43132     8.3126  -13.7439   47.0
경기    시흥     53.1303  92.2638   -10.1613    3.22646  -13.3878   48.0
대전    서구     40.1599  79.2348   -23.1318   -9.80258  -13.3292   49.0
충남    천안     35.7943  74.1313   -27.4974    -14.906  -12.5913   50.0
서울    금천구    65.5571  102.498    2.26542    13.4603  -11.1949   51.0
      동작구    84.6636  120.865     21.372    31.8277  -10.4557   52.0
      도봉구    58.4194  94.2227   -4.87222    5.18534  -10.0576   53.0
경기    이천     68.9247  102.385    5.63309    13.3475  -7.71445   54.0
인천    서구     40.2375  73.1186   -23.0542   -15.9187  -7.13544   55.0
      인천     34.4473  66.5851   -28.8443   -22.4523  -6.39206   56.0
경기    평택     31.1615  63.0587   -32.1301   -25.9787   -6.1514   57.0
      팔달구    

In [4]:
!open .

In [22]:
sorted_df.index

MultiIndex(levels=[['5개광역시', '6개광역시', '강원', '경기', '경남', '경북', '광주', '구분', '기타지방', '대구', '대전', '부산', '서울', '세종', '수도권', '울산', '인천', '전국', '전남', '전북', '제주도', '충남', '충북'], ['5개광역시', '6개광역시', '강남', '강남구', '강동구', '강릉', '강북', '강북구', '강서구', '강원', '거제', '경기', '경남', '경북', '경산', '계룡', '계양구', '고양', '공주', '과천', '관악구', '광명', '광산구', '광양', '광주', '광진구', '구로구', '구리', '구미', '구분', '군산', '군포', '권선구', '금정구', '금천구', '기장군', '기타지방', '기흥구', '김천', '김포', '김해', '남구', '남동구', '남양주', '노원구', '논산', '단원구', '달서구', '달성군', '당진', '대구', '대덕구', '대전', '덕양구', '덕진구', '도봉구', '동구', '동남구', '동대문구', '동두천', '동래구', '동안구', '동작구', '마산합포구', '마산회원구', '마포구', '만안구', '목포', '미추홀구', '부산', '부산진구', '부천', '부평구', '북구', '분당구', '사상구', '사하구', '상당구', '상록구', '서구', '서귀포', '서대문구', '서북구', '서산', '서울', '서원구', '서초구', '성남', '성동구', '성북구', '성산구', '세종', '송파구', '수도권', '수성구', '수영구', '수원', '수정구', '수지구', '순천', '시흥', '아산', '안동', '안산', '안성', '안양', '양산', '양주', '양천구', '여수', '연수구', '연제구', '영도구', '영등포구', '영통구', '오산', '완산구', '용산구', '용인', '울산', '울주군', '원주', '유성구', '은평구', '의

In [4]:
print('test')

test
